# **Nbeats Hourly Train***

# N-BEATS Model with M4 Hourly Train Dataset



In [ ]:
#@title **Installations**
import subprocess
import sys

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

packages = ["darts", "optuna"]

for package in packages:
    try:
        __import__(package)
        print(f"{package} is installed")
    except ImportError:
        print(f"{package} is not installed. Installing now...")
        install(package)

print("Success")


In [ ]:
# @title Necessary Imports

# Basic imports
import numpy as np
import pandas as pd
from typing import Union, List
import time
import gc

# Data visualization imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
sns.set_style('whitegrid')
import matplotlib.pyplot as plt
from IPython.display import display
%matplotlib inline
from tqdm import tqdm

# Machine learning imports
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import statsmodels.api as sm
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

# Optuna imports
import optuna
from optuna.integration import PyTorchLightningPruningCallback
import optuna.visualization as ovis

# Darts imports
from darts import TimeSeries
from darts.models import NaiveSeasonal, NaiveDrift, ExponentialSmoothing, VARIMA, AutoARIMA
from darts.models import RNNModel, NBEATSModel, NLinearModel, DLinearModel, LightGBMModel, NaiveMean, NaiveSeasonal, NaiveDrift, LinearRegressionModel, RandomForest
from darts.dataprocessing.transformers import Scaler, MissingValuesFiller, InvertibleMapper
from darts.metrics import mape, smape, mae, r2_score, mse, smape, mase,rmse, rmsle
from darts.utils.timeseries_generation import (gaussian_timeseries,linear_timeseries,sine_timeseries)
from darts.dataprocessing import Pipeline

# Drive mount
from google.colab import drive
drive.mount('/content/drive')     #Drive abcr5914


##**Required Functions**
- **fit_predict_and_evaluate:** This function fits the model to the training data, makes predictions, and evaluates the model's performance using Mean Absolute Error (MAE) and Symmetric Mean Absolute Percentage Error (sMAPE). This function encapsulates the entire process of training, predicting, and evaluating a model.
```python
fit_predict_and_evaluate(model, training_set, validation_set, n_forecasts, past_covariates_train = None,
                                past_covariates_validation = None, series_list=False, model_name='Model'):
```

- **Cross Validation Functions**:
  - Group TimeSeries Split Cross Validation
  - Purged Group TimeSeries Split Cross Validation
  - Combinatorial Purged Group TimeSeries Split Cross Validation

- **Optuna Hyperparameter Otimization**
  - LightGBM Otimization
  - Nbeats Optimization

- **Plotting Functions**
  ```
  plot_lists(train_set, validation_set, prediction_set, len = 700)
  ```

- **WAPE Error Metric**



In [ ]:
#@title Required functions

def fit_predict_and_evaluate(model, training_set, validation_set, n_forecasts, past_covariates_train = None, past_covariates_validation = None, series_list=False, model_name='Model'):
    """
    Fit the model to the training data, make predictions, and evaluate the metrics.

    Args:
        model: The model to be trained and used for predictions.
        training_set: The data to train the model.
        validation_set: The data to validate the model.
        n_forecasts (int): The number of forecast steps.
        past_covariates_train: The past covariates for training.
        past_covariates_validation: The past covariates for validation.
        series_list (bool, optional): If True, the training_set will be passed as a series to the predict method. Defaults to False.
        model_name (str, optional): The name of the model. Defaults to 'Model'.

    Returns:
        dict: A dictionary containing the forecast and the metrics.

    Raises:
        ValueError: If the lengths of actual_values and predicted_values are not the same.
    """
    # Validate inputs
    if not model or not training_set or n_forecasts < 1:
        raise ValueError("Invalid inputs provided.")

    # Fit the model and make predictions
    try:
        train_start = time.time()
        model.fit(training_set, past_covariates=past_covariates_train)
        train_stop = time.time()
        forecast_start = time.time()
        forecast = model.predict(n_forecasts, series=training_set if series_list else None, past_covariates=past_covariates_validation, show_warnings=False)
        forecast_stop = time.time()
    except Exception as e:
        print(f"An error occurred while fitting the model or making predictions: {e}")
        return None

    # Validate the forecast and actual values length
    #if len(validation_set) != len(forecast):
     #   raise ValueError("Lengths of validation_set and forecast must be the same.")


    #mae_value = mae(validation_set, forecast, inter_reduction = np.mean)
    smape_value = smape(validation_set, forecast, inter_reduction = np.mean)
    smape_list = list(smape(validation_set, forecast))
    #rmse_value = rmse(validation_set, forecast, inter_reduction = np.mean)
    wape_value = (wape(validation_set, forecast, inter_reduction= np.mean))
    wape_list = list(wape(validation_set, forecast))

    train_time = train_stop-train_start
    forecast_time = forecast_stop - forecast_start


    print('-----------------------------------------')
    #print(f"MAE ({model_name}) = {mae_value:.2f}")
    #print(f"sMAPE ({model_name}) = {smape_value:.2f}")
    print(f"WAPE ({model_name}) = {wape_value:.2f}")
    #print(f"RMSE ({model_name}) = {rmse_value:.2f}")
    print(f"Training Time ({model_name}) = {train_time:.2f}sec")
    print(f"Forecast Time ({model_name}) = {forecast_time:.2f}sec")
    print(f"Total Time ({model_name}) = {train_time+forecast_time:.2f}sec")
    print('-----------------------------------------')

    # 'MAE': mae_value, 'sMAPE': smape_value, 'RMSE': rmse_value, , 'Total Time': train_time + forecast_time

    return {'forecast': forecast, 'wape': wape_list,'smape': smape_list, 'Training Time': train_time, 'Forecast Time': forecast_time}


In [ ]:
#@title WAPE Error Metric

# WAPE ERROR METRIC
from functools import wraps
from inspect import signature
from typing import Callable, Optional, Sequence, Tuple, Union
from warnings import warn
from darts import TimeSeries
from darts.logging import get_logger, raise_if_not, raise_log
from darts.utils import _build_tqdm_iterator, _parallel_apply
from darts.utils.statistics import check_seasonality
from darts.logging import get_logger, raise_if_not, raise_log
logger = get_logger(__name__)



def multi_ts_support(func):
    """
    This decorator further adapts the metrics that took as input two univariate/multivariate ``TimeSeries`` instances,
    adding support for equally-sized sequences of ``TimeSeries`` instances. The decorator computes the pairwise metric
    for ``TimeSeries`` with the same indices, and returns a float value that is computed as a function of all the
    pairwise metrics using a `inter_reduction` subroutine passed as argument to the metric function.

    If a 'Sequence[TimeSeries]' is passed as input, this decorator provides also parallelisation of the metric
    evaluation regarding different ``TimeSeries`` (if the `n_jobs` parameter is not set 1).
    """

    @wraps(func)
    def wrapper_multi_ts_support(*args, **kwargs):
        actual_series = (
            kwargs["actual_series"] if "actual_series" in kwargs else args[0]
        )
        pred_series = (
            kwargs["pred_series"]
            if "pred_series" in kwargs
            else args[0]
            if "actual_series" in kwargs
            else args[1]
        )

        n_jobs = kwargs.pop("n_jobs", signature(func).parameters["n_jobs"].default)
        verbose = kwargs.pop("verbose", signature(func).parameters["verbose"].default)

        raise_if_not(isinstance(n_jobs, int), "n_jobs must be an integer")
        raise_if_not(isinstance(verbose, bool), "verbose must be a bool")

        actual_series = (
            [actual_series]
            if not isinstance(actual_series, Sequence)
            else actual_series
        )
        pred_series = (
            [pred_series] if not isinstance(pred_series, Sequence) else pred_series
        )

        raise_if_not(
            len(actual_series) == len(pred_series),
            "The two TimeSeries sequences must have the same length.",
            logger,
        )

        num_series_in_args = int("actual_series" not in kwargs) + int(
            "pred_series" not in kwargs
        )
        kwargs.pop("actual_series", 0)
        kwargs.pop("pred_series", 0)

        iterator = _build_tqdm_iterator(
            iterable=zip(actual_series, pred_series),
            verbose=verbose,
            total=len(actual_series),
        )

        value_list = _parallel_apply(
            iterator=iterator,
            fn=func,
            n_jobs=n_jobs,
            fn_args=args[num_series_in_args:],
            fn_kwargs=kwargs,
        )

        # in case the reduction is not reducing the metrics sequence to a single value, e.g., if returning the
        # np.ndarray of values with the identity function, we must handle the single TS case, where we should
        # return a single value instead of a np.array of len 1

        if len(value_list) == 1:
            value_list = value_list[0]

        if "inter_reduction" in kwargs:
            return kwargs["inter_reduction"](value_list)
        else:
            return signature(func).parameters["inter_reduction"].default(value_list)

    return wrapper_multi_ts_support



def multivariate_support(func):
    """
    This decorator transforms a metric function that takes as input two univariate TimeSeries instances
    into a function that takes two equally-sized multivariate TimeSeries instances, computes the pairwise univariate
    metrics for components with the same indices, and returns a float value that is computed as a function of all the
    univariate metrics using a `reduction` subroutine passed as argument to the metric function.
    """

    @wraps(func)
    def wrapper_multivariate_support(*args, **kwargs):
        # we can avoid checks about args and kwargs since the input is adjusted by the previous decorator
        actual_series = args[0]
        pred_series = args[1]

        raise_if_not(
            actual_series.width == pred_series.width,
            "The two TimeSeries instances must have the same width.",
            logger,
        )

        value_list = []
        for i in range(actual_series.width):
            value_list.append(
                func(
                    actual_series.univariate_component(i),
                    pred_series.univariate_component(i),
                    *args[2:],
                    **kwargs
                )
            )  # [2:] since we already know the first two arguments are the series
        if "reduction" in kwargs:
            return kwargs["reduction"](value_list)
        else:
            return signature(func).parameters["reduction"].default(value_list)

    return wrapper_multivariate_support

def _get_values(
    series, stochastic_quantile = 0.5):
    """
    Returns the numpy values of a time series.
    For stochastic series, return either all sample values with (stochastic_quantile=None) or the quantile sample value
    with (stochastic_quantile {>=0,<=1})
    """
    if series.is_deterministic:
        series_values = series.univariate_values()
    else:  # stochastic
        if stochastic_quantile is None:
            series_values = series.all_values(copy=False)
        else:
            series_values = series.quantile_timeseries(
                quantile=stochastic_quantile
            ).univariate_values()
    return series_values

def _get_values_or_raise(
    series_a,
    series_b,
    intersect,
    stochastic_quantile = 0.5,
    remove_nan_union: bool = False):
    """Returns the processed numpy values of two time series. Processing can be customized with arguments
    `intersect, stochastic_quantile, remove_nan_union`.

    Raises a ValueError if the two time series (or their intersection) do not have the same time index.

    Parameters
    ----------
    series_a
        A univariate deterministic ``TimeSeries`` instance (the actual series).
    series_b
        A univariate (deterministic or stochastic) ``TimeSeries`` instance (the predicted series).
    intersect
        A boolean for whether to only consider the time intersection between `series_a` and `series_b`
    stochastic_quantile
        Optionally, for stochastic predicted series, return either all sample values with (`stochastic_quantile=None`)
        or any deterministic quantile sample values by setting `stochastic_quantile=quantile` {>=0,<=1}.
    remove_nan_union
        By setting `remove_non_union` to True, remove all indices from `series_a` and `series_b` which have a NaN value
        in either of the two input series.
    """

    raise_if_not(
        series_a.width == series_b.width,
        "The two time series must have the same number of components",
        logger,
    )

    raise_if_not(isinstance(intersect, bool), "The intersect parameter must be a bool")

    series_a_common = series_a.slice_intersect(series_b) if intersect else series_a
    series_b_common = series_b.slice_intersect(series_a) if intersect else series_b

    raise_if_not(
        series_a_common.has_same_time_as(series_b_common),
        "The two time series (or their intersection) "
        "must have the same time index."
        "\nFirst series: {}\nSecond series: {}".format(
            series_a.time_index, series_b.time_index
        ),
        logger,
    )

    series_a_det = _get_values(series_a_common, stochastic_quantile=stochastic_quantile)
    series_b_det = _get_values(series_b_common, stochastic_quantile=stochastic_quantile)

    if not remove_nan_union:
        return series_a_det, series_b_det

    b_is_deterministic = bool(len(series_b_det.shape) == 1)
    if b_is_deterministic:
        isnan_mask = np.logical_or(np.isnan(series_a_det), np.isnan(series_b_det))
    else:
        isnan_mask = np.logical_or(
            np.isnan(series_a_det), np.isnan(series_b_det).any(axis=2).flatten()
        )
    return np.delete(series_a_det, isnan_mask), np.delete(
        series_b_det, isnan_mask, axis=0
    )
@multi_ts_support
@multivariate_support
def wape(
    actual_series,
    pred_series,
    reduction = np.mean,
    inter_reduction = lambda x:x,
    intersect = True,
    n_jobs = 1,
    verbose = False):

    y_true, y_hat = _get_values_or_raise(
        actual_series, pred_series, intersect, remove_nan_union=True
    )
    raise_if_not(
        np.logical_or(y_true != 0, y_hat != 0).all(),
        "The actual series must be strictly positive to compute the WAPE.",
        logger,
    )
    return np.sum(np.abs(y_true - y_hat)) / np.sum(y_true)

@multi_ts_support
@multivariate_support
def smape_100(
    actual_series,
    pred_series,
    intersect = True,
    *,
    reduction = np.mean,
    inter_reduction = lambda x: x,
    n_jobs = 1,
    verbose = False):

    y_true, y_hat = _get_values_or_raise(
        actual_series, pred_series, intersect, remove_nan_union=True
    )
    raise_if_not(
        np.logical_or(y_true != 0, y_hat != 0).all(),
        "The actual series must be strictly positive to compute the sMAPE.",
        logger,
    )
    return 100.0 * np.mean(np.abs(y_true - y_hat) / (np.abs(y_true) + np.abs(y_hat)))

In [ ]:
#@title Cross validation functions
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args
#(n_splits=(default 5), max_train_size = (default = None))
# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class GroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_size : int, default=None
        Maximum size for a single training set.
    Examples
    --------
    >>> import numpy as np
    >>> from sklearn.model_selection import GroupTimeSeriesSplit
    >>> groups = np.array(['a', 'a', 'a', 'a', 'a', 'a',\
                           'b', 'b', 'b', 'b', 'b',\
                           'c', 'c', 'c', 'c',\
                           'd', 'd', 'd'])
    >>> gtss = GroupTimeSeriesSplit(n_splits=3)
    >>> for train_idx, test_idx in gtss.split(groups, groups=groups):
    ...     print("TRAIN:", train_idx, "TEST:", test_idx)
    ...     print("TRAIN GROUP:", groups[train_idx],\
                  "TEST GROUP:", groups[test_idx])
    TRAIN: [0, 1, 2, 3, 4, 5] TEST: [6, 7, 8, 9, 10]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a']\
    TEST GROUP: ['b' 'b' 'b' 'b' 'b']
    TRAIN: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10] TEST: [11, 12, 13, 14]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a' 'b' 'b' 'b' 'b' 'b']\
    TEST GROUP: ['c' 'c' 'c' 'c']
    TRAIN: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]\
    TEST: [15, 16, 17]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a' 'b' 'b' 'b' 'b' 'b' 'c' 'c' 'c' 'c']\
    TEST GROUP: ['d' 'd' 'd']
    """
    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_size=None
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_size = max_train_size

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))
        group_test_size = n_groups // n_folds
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []
            for train_group_idx in unique_groups[:group_test_start]:
                train_array_tmp = group_dict[train_group_idx]
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)
            train_end = train_array.size
            if self.max_train_size and self.max_train_size < train_end:
                train_array = train_array[train_end -
                                          self.max_train_size:train_end]
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)
            yield [int(i) for i in train_array], [int(i) for i in test_array]

class PurgedGroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Allows for a gap in groups to avoid potentially leaking info from
    train into test if the model has windowed or lag features.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    group_gap : int, default=None
        Gap between train and test
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split
    """

    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_group_size=np.inf,
                 max_test_group_size=np.inf,
                 group_gap=None,
                 verbose=False
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.group_gap = group_gap
        self.max_test_group_size = max_test_group_size
        self.verbose = verbose

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        group_gap = self.group_gap
        max_test_group_size = self.max_test_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_test_size = min(n_groups // n_folds, max_test_group_size)
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []

            group_st = max(0, group_test_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_test_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]

                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

            train_end = train_array.size

            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)

            test_array  = test_array[group_gap:]


            if self.verbose > 0:
                    pass

            yield [int(i) for i in train_array], [int(i) for i in test_array]

import numpy as np
from scipy.special import comb
from itertools import combinations

class CombinatorialPurgedGroupKFold():
    def __init__(self, n_splits = 6, n_test_splits = 2, purge = 1, pctEmbargo = 0.01, **kwargs):
        self.n_splits = n_splits
        self.n_test_splits = n_test_splits
        self.purge = purge
        self.pctEmbargo = pctEmbargo

    def split(self, X, y = None, groups = None):
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")

        u, ind = np.unique(groups, return_index = True)
        unique_groups = u[np.argsort(ind)]
        n_groups = len(unique_groups)
        group_dict = {}
        for idx in range(len(X)):
            if groups[idx] in group_dict:
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]

        n_folds = comb(self.n_splits, self.n_test_splits, exact = True)
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        mbrg = int(n_groups * self.pctEmbargo)
        if mbrg < 0:
            raise ValueError(
                "The number of 'embargoed' groups should not be negative")

        split_dict = {}
        group_test_size = n_groups // self.n_splits
        for split in range(self.n_splits):
            if split == self.n_splits - 1:
                split_dict[split] = unique_groups[int(split * group_test_size):].tolist()
            else:
                split_dict[split] = unique_groups[int(split * group_test_size):int((split + 1) * group_test_size)].tolist()

        for test_splits in combinations(range(self.n_splits), self.n_test_splits):
            test_groups = []
            banned_groups = []
            for split in test_splits:
                test_groups += split_dict[split]
                banned_groups += unique_groups[split_dict[split][0] - self.purge:split_dict[split][0]].tolist()
                banned_groups += unique_groups[split_dict[split][-1] + 1:split_dict[split][-1] + self.purge + mbrg + 1].tolist()
            train_groups = [i for i in unique_groups if (i not in banned_groups) and (i not in test_groups)]

            train_idx = []
            test_idx = []
            for train_group in train_groups:
                train_idx += group_dict[train_group]
            for test_group in test_groups:
                test_idx += group_dict[test_group]
            yield train_idx, test_idx

In [ ]:
#@title Plotting Functions
def plot_lists(train_set, validation_set, prediction_set, len = 700):
    plt.figure(figsize=(10, 3))

    # Use the time index from the Darts TimeSeries objects
    plt.plot(train_set.time_index, train_set.values(), color='blue', label='Train Set')
    plt.plot(validation_set.time_index, validation_set.values(), color='green', label='Validation Set')
    plt.plot(prediction_set.time_index, prediction_set.values(), color='red', label='Prediction Set')

    for i in range(24, len, 24):
        plt.axvline(x=i, color='gray', linestyle='--')
    plt.legend(loc='lower center', bbox_to_anchor=(0.5, 0.96), ncol=3)

    plt.xlim(0,len)
    plt.ylim(0, 1) # Scaled Values
    plt.xlabel('Time')
    plt.ylabel('Observations')
    plt.title(f'TimeSeries Plot', y = 1.08)
    plt.show()


In [ ]:
#@title **Model Creation Functions**

def create_nbeats_model(input_chunk_length=24, output_chunk_length=24, generic_architecture=True,
               num_stacks=4, num_blocks=2, num_layers=2, layer_widths=16,
               n_epochs=2, nr_epochs_val_period=1, batch_size=32, model_name="NBEATS"):
    """
    Create and return an NBEATSModel with the specified parameters.

    Args:
        input_chunk_length (int): The length of the input sequence.
        output_chunk_length (int): The length of the output sequence.
        generic_architecture (bool): Whether to use the generic architecture.
        num_stacks (int): The number of stacks.
        num_blocks (int): The number of blocks per stack.
        num_layers (int): The number of layers per block.
        layer_widths (int): The number of units per layer.
        n_epochs (int): The number of epochs.
        nr_epochs_val_period (int): The number of epochs between each validation set evaluation.
        batch_size (int): The size of the batch.
        model_name (str): The name of the model.

    Returns:
        NBEATSModel: A trained NBEATSModel.
    """
    model_nbeats = NBEATSModel(
        input_chunk_length=input_chunk_length,
        output_chunk_length=output_chunk_length,
        generic_architecture=generic_architecture,
        num_stacks=num_stacks,
        num_blocks=num_blocks,
        num_layers=num_layers,
        layer_widths=layer_widths,
        n_epochs=n_epochs,
        nr_epochs_val_period=nr_epochs_val_period,
        batch_size=batch_size,
        model_name=model_name
    )

    return model_nbeats


def create_lightgbm_model(lags=None,
                          output_chunk_length=2,
                          max_depth=3,
                          n_estimators=50,
                          learning_rate=0.1,
                          verbose =-1,
                          use_static_covariates=True):
    """
    Creates an instance of LightGBMModel with the specified parameters.

    Parameters:
    lags (int): Number of lags to use in the model.
    output_chunk_length (int): Length of the output chunks.
    max_depth (int): Maximum depth of the trees.
    n_estimators (int): Number of trees in the model.
    learning_rate (float): Learning rate for the model.

    Returns:
    model (LightGBMModel): An instance of LightGBMModel.
    """

    model = LightGBMModel(
        lags=lags,
        use_static_covariates = use_static_covariates,
        output_chunk_length=output_chunk_length,
        max_depth=max_depth,
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        verbose=verbose,
    )

    return model


#**Forecasting**

In [ ]:
#@title Loading the Dataset
base_path = '/content/drive/MyDrive/Master_Thesis/Datasets/m4/'

m4_hour = pd.read_csv(base_path + 'Train/Hourly-train.csv', index_col=0, header=None).drop(index = ['V1'])

# Load and process M4_info
M4_info = pd.read_csv(f'{base_path}M4-info.csv')
hour_info = M4_info[M4_info['M4id'].str.startswith('H')].set_index('M4id')

m4_hourly = pd.merge(hour_info, m4_hour, left_index=True, right_index=True).drop(columns = ['category','Frequency','Horizon','SP']).reset_index()
m4_hourly.rename(columns={'index': 'TimeSeries'}, inplace=True)
print('m4_hourly')
display(m4_hourly.head(3))

# Melting m4_hourly
m4_hourly_melt = m4_hourly.melt(id_vars=['TimeSeries','StartingDate'], var_name='Time', value_name='Observation').dropna()

# Creating a dictionary that maps each unique 'H' value and StartingDate to unique integers
label_dict = {k: v for v, k in enumerate(m4_hourly_melt['TimeSeries'].unique())}
m4_hourly_melt['TimeSeries_id'] = m4_hourly_melt['TimeSeries'].map(label_dict)

label_dict = {k: v for v, k in enumerate(m4_hourly_melt['StartingDate'].unique())}
m4_hourly_melt['StartingDate_id'] = m4_hourly_melt['StartingDate'].map(label_dict)

print('\nm4_hourly_melted')
display(m4_hourly_melt.head(3))

In [ ]:
#@title TimeSeries Conversion

ts_hourly = TimeSeries.from_group_dataframe(
                                m4_hourly_melt,
                                group_cols = ['TimeSeries_id'],
                                time_col = 'Time',
                                value_cols = 'Observation',
                                static_cols = 'StartingDate_id'
                                )

ts_hourly_scaled, training_set, validation_set = [],[],[]

split_point = 24

for i in range(414):
  scaler = Scaler()
  ts_hourly_scaled.append(scaler.fit_transform(ts_hourly[i]))

  train, val = ts_hourly_scaled[i][:-split_point], ts_hourly_scaled[i][-split_point:]

  training_set.append(train)
  validation_set.append(val)


In [ ]:
#@title LightGBM
model_lgbm = create_lightgbm_model(lags = [-1,-24,-120,-168], output_chunk_length=2, max_depth=3,
                       n_estimators=50, learning_rate=0.1, verbose=-1)


results = fit_predict_and_evaluate(
                      model=model_lgbm,
                      training_set=training_set,
                      validation_set=validation_set,
                      n_forecasts=split_point,
                      series_list=True,
                      model_name='LightGBM')

prediction_set = results['forecast']

'''
# Plots

i = 140
plot_lists(training_set[i], validation_set[i], prediction_set[i],700)

i = 245
plot_lists(training_set[i], validation_set[i], prediction_set[i],960)
'''

/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function delayed is deprecated; The function `delayed` has been moved from `sklearn.utils.fixes` to `sklearn.utils.parallel`. This import path will be removed in 1.5.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/parallel.py:114: UserWarning: `sklearn.utils.parallel.delayed` should be used with `sklearn.utils.parallel.Parallel` to make it possible to propagate the scikit-learn configuration of the current thread to the joblib workers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function delayed is deprecated; The function `delayed` has been moved from `sklearn.utils.fixes` to `sklearn.utils.parallel`. This import path will be removed in 1.5.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/parallel.py:114: UserWarning: `sklearn.utils.para

KeyboardInterrupt: 

In [ ]:
#@title **Fit_Predict_Evaluate**

models = {

  # Regression Models
  'Linear Regression Model' : LinearRegressionModel(lags = [-1,-24,-120,-168]),

  # traditional ML Models
  'Random Forest Model' : RandomForest(lags = [-1,-24,-120,-168]),
  'LightGBM Model': create_lightgbm_model(lags = [-1,-24,-120,-168]),

  # Deep Neural Networks
  'RNN Model' : RNNModel(input_chunk_length=24 ,model='RNN',n_epochs=5),
  'NBeats Model' : create_nbeats_model()

  }

results_list = {}
metrics_list = {}

for name, model in models.items():

  print('\n',name)
  result = fit_predict_and_evaluate(
                    model = model,
                    training_set = training_set,
                    validation_set = validation_set,
                    n_forecasts = split_point,
                    series_list = True,
                    model_name = name)

  results_list[f'{name}_results'] = result
  metrics_list[f'{name}_metrics']  = {'WAPE':np.mean(result['wape']) ,'Training Time': result['Training Time'],'Forecast Time': result['Forecast Time']}


# Convert results_list to a DataFrame
results_df = pd.DataFrame.from_dict(metrics_list, orient='index')


In [ ]:
results_df

##**Optuna**

In [ ]:
def objective(trial, training_data, validation_data, model_type):
    """
    Optuna objective function for hyperparameter optimization.
    This function is to find the best parameters for a model
    to minimize the Weighted Absolute Percentage Error (sMAPE)
    between predicted and actual values in a validation dataset.
    """
    # Define hyperparameters


    # Additional hyperparameters for specific models
    if model_type == 'NBEATS':
        input_chunk_length = trial.suggest_categorical('input_chunk_length', [24, 48])
        batch_size = trial.suggest_categorical('batch_size', [8, 16])
        output_chunk_length = trial.suggest_categorical('output_chunk_length', [24, 48])
        n_epochs = trial.suggest_int('n_epochs', 1, 4)
        num_stacks = trial.suggest_int('num_stacks', 1, 3)
        num_blocks = trial.suggest_int('num_blocks', 1, 3)
        num_layers = trial.suggest_int('num_layers', 1, 3)
        layer_widths = trial.suggest_categorical('layer_widths', [16, 32])
        dropout = trial.suggest_float('dropout', 0.0, 0.2)
        activation = trial.suggest_categorical('activation', ['ReLU', 'RReLU', 'LeakyReLU'])
        model = create_nbeats_model(
            input_chunk_length=input_chunk_length,
            output_chunk_length=output_chunk_length,
            generic_architecture=True,
            num_stacks=num_stacks,
            num_blocks=num_blocks,
            num_layers=num_layers,
            layer_widths=layer_widths,
            n_epochs=n_epochs,
            nr_epochs_val_period=1,
            batch_size=batch_size,
            model_name="NBEATS"
        )

    elif model_type == 'LightGBM':
        #lags_list1 = [-1, -24, -120, -168]
        #lags_list2 = [-2, -48, -240, -336]
        #lags = trial.suggest_categorical('lags', [lags_list1, lags_list2]).
        output_chunk_length = trial.suggest_int('output_chunk_length', 1, 4)
        max_depth = trial.suggest_int('max_depth', 1, 6)
        n_estimators = trial.suggest_int('n_estimators', 10, 100)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.1)

        model = create_lightgbm_model(
          lags=[-1, -24, -120, -168],
          output_chunk_length=output_chunk_length,
          max_depth=max_depth,
          n_estimators=n_estimators,
          learning_rate=learning_rate,
          verbose=-1,
          use_static_covariates=True
      )

    else:
        raise ValueError("Invalid model type")

    # Predict and calculate WAPE
    forecast = fit_predict_and_evaluate(
        model=model,
        training_set=training_data,
        validation_set=validation_data,
        n_forecasts=28,
        series_list=True,
        model_name = model_type
    )
    wape_result = np.mean(forecast['wape'])

    return wape_result

In [ ]:
study_LGBM = optuna.create_study(direction='minimize')
study_LGBM.optimize(lambda trial: objective(trial, training_data = training_set, validation_data = validation_set, model_type = 'LightGBM'), n_trials=10)

print('Best Params for LightGBM:')
trial_LGBM = study_LGBM.best_trial
print(trial_LGBM.params)

In [ ]:
# plots Optuna
display(optuna.visualization.plot_optimization_history(study_LGBM))
display(optuna.visualization.plot_param_importances(study_LGBM))
display(optuna.visualization.plot_parallel_coordinate(study_LGBM))

In [ ]:
study_NB = optuna.create_study(direction='minimize')
study_NB.optimize(lambda trial: objective(trial, training_data = training_set, validation_data = validation_set, model_type = 'NBEATS'), n_trials=10)

print('Best Params for NBEATS:')
trial_NB = study_NB.best_trial
print(trial_NB.params)

In [ ]:
# plots Optuna
display(optuna.visualization.plot_optimization_history(study_NB))
display(optuna.visualization.plot_param_importances(study_NB))
display(optuna.visualization.plot_parallel_coordinate(study_NB))

## **Cross Validation**

In [ ]:
#@title Model List for Cross Validation
# uncomment the models required
models = {

  # Regression Models
  #'Linear Regression Model' : LinearRegressionModel(lags = [-1,-24,-120,-168]),

  # traditional ML Models
  #'Random Forest Model' : RandomForest(lags = [-1,-24,-120,-168]),
  'LightGBM Model': create_lightgbm_model(lags = [-1,-24,-120,-168], output_chunk_length=2, max_depth=3, n_estimators=50, learning_rate=0.1, verbose=-1),

  # Deep Neural Networks
  #'RNN Model' : RNNModel(input_chunk_length=24 ,model='RNN',n_epochs=5),
  #'NBeats Model' : create_nbeats_model()

  }

In [ ]:
#@title Walk forward Cross Validation
train_split = []
validation_split = []

WAPE_dict = {}

n_splits = 3   # -> default: 5

tscv = TimeSeriesSplit(n_splits = n_splits)

for i in range(len(ts_hourly)):
#for i in range(4):
  train_i = []
  val_i = []

  for split, (train, test) in enumerate(tscv.split(ts_hourly_scaled[i])):
    train, val = ts_hourly_scaled[i][train[0]:train[-1]], ts_hourly_scaled[i][test[0]:test[-1]]

    train_i.append(train)
    val_i.append(val)
  train_split.append(train_i)
  validation_split.append(val_i)

# Splitting the train and test data splits into lists, one for each split
train_splits_tscv = [[x for x in group] for group in zip(*train_split)]
validation_splits_tscv =  [[x for x in group] for group in zip(*validation_split)]

results_list_wfcv = {}
metrics_list_wfcv = {}

for name, model in models.items():

  for training_set, validation_set, n_split in zip(train_splits_tscv, validation_splits_tscv,range(n_splits)):
    n_forecasts = len(validation_set[1])
    print('\n',name)
    result = fit_predict_and_evaluate(
                                  model = model,
                                  training_set = training_set,
                                  validation_set = validation_set,
                                  n_forecasts = n_forecasts,
                                  series_list = True,
                                  model_name = name)

    results_list_wfcv[f'{name}_results'] = result
    metrics_list_wfcv[f'{name}_metrics']  = {'WAPE':np.mean(result['wape']) ,'Training Time': result['Training Time'],'Forecast Time': result['Forecast Time'], 'split': n_split}


In [ ]:
'''
#lightgbm

model_lgbm = create_lightgbm_model(lags = [-1,-24,-120,-168] ,  n_estimators = 96, learning_rate = 0.048170614847117356, output_chunk_length=2, max_depth=5,
                        verbose=-1)
i = 245
plot_splits_tscv = []
pred = []
for training_set, validation_set in zip(train_splits_tscv, validation_splits_tscv):
  n_forecasts=len(validation_set[1])

  results = fit_predict_and_evaluate(
                        model=model_lgbm,
                        training_set=training_set,
                        validation_set=validation_set,
                        n_forecasts=n_forecasts,
                        series_list=True,
                        model_name='LightGBM')

  pred.append(results['forecast'])
  splits = [training_set[i], validation_set[i],results['forecast'][i]]
  plot_splits_tscv.append(splits)
'''
print(' ')

In [ ]:
#@title Rolling Cross Validation
train_split = []
validation_split = []

WAPE_dict = {}

n_splits = 3    # -> default: 5

ts_rcv = TimeSeriesSplit(n_splits = 3, max_train_size = 300 )

for i in range(len(ts_hourly)):
#for i in range(4):
  train_i = []
  val_i = []

  for split, (train, test) in enumerate(ts_rcv.split(ts_hourly_scaled[i])):
    train, val = ts_hourly_scaled[i][train[0]:train[-1]], ts_hourly_scaled[i][test[0]:test[-1]]

    train_i.append(train)
    val_i.append(val)
  train_split.append(train_i)
  validation_split.append(val_i)



# Splitting the train and test data splits into lists, one for each split
train_splits_rcv = [[x for x in group] for group in zip(*train_split)]
validation_splits_rcv =  [[x for x in group] for group in zip(*validation_split)]

results_list_rcv = {}
metrics_list_rcv = {}

for name, model in models.items():

  for training_set, validation_set, n_split in zip(train_splits_rcv, validation_splits_rcv,range(n_splits)):
    n_forecasts = len(validation_set[1])
    print('\n',name)
    result = fit_predict_and_evaluate(
                                  model = model,
                                  training_set = training_set,
                                  validation_set = validation_set,
                                  n_forecasts = n_forecasts,
                                  series_list = True,
                                  model_name = name)

    results_list_rcv[f'{name}_results'] = result
    metrics_list_rcv[f'{name}_metrics']  = {'WAPE':np.mean(result['wape']) ,'Training Time': result['Training Time'],'Forecast Time': result['Forecast Time'], 'split': n_split}

In [ ]:
'''
#for now just lightgbm

model_lgbm = create_lightgbm_model(lags = [-1,-24,-120,-168], output_chunk_length=2, max_depth=3,
                       n_estimators=50, learning_rate=0.1, verbose=-1)
i = 245
plot_splits_rcv = []
pred_rcv = []
for training_set, validation_set in zip(train_splits_rcv, validation_splits_rcv):
  n_forecasts=len(validation_set[1])

  results = fit_predict_and_evaluate(
                        model=model_lgbm,
                        training_set=training_set,
                        validation_set=validation_set,
                        n_forecasts=n_forecasts,
                        series_list=True,
                        model_name='LightGBM')

  pred_rcv.append(results['forecast'])
  splits = [training_set[i], validation_set[i],results['forecast'][i]]
  plot_splits_rcv.append(splits)
  '''
print(' ')

In [ ]:
#@title Group Cross Validation

df = m4_hourly_melt.copy()
df['Group'] = (df.groupby('TimeSeries').cumcount() // 24) + 1
n_splits = 3
gscv = GroupTimeSeriesSplit(n_splits=n_splits)
day_groups = df['TimeSeries_id'].unique()
train_split = []
validation_split = []
for i in day_groups:
  group_data = df[df['TimeSeries_id'] == i].reset_index(drop=True)

  X_group = group_data
  sub_groups = group_data['Group']

  train_i = []
  val_i = []

  for train_idx, test_idx in gscv.split(X_group, groups=sub_groups):
    train, val = ts_hourly_scaled[i][train_idx[0]:train_idx[-1]], ts_hourly_scaled[i][test_idx[0]:test_idx[-1]]

    train_i.append(train)
    val_i.append(val)
  train_split.append(train_i)
  validation_split.append(val_i)


# Splitting the train and test data splits into lists, one for each split
train_splits_gscv = [[x for x in group] for group in zip(*train_split)]
validation_splits_gscv =  [[x for x in group] for group in zip(*validation_split)]

results_list_gscv = {}
metrics_list_gscv = {}

for name, model in models.items():

  for training_set, validation_set, n_split in zip(train_splits_gscv, validation_splits_gscv,range(n_splits)):
    n_forecasts = len(validation_set[1])
    print('\n',name)
    result = fit_predict_and_evaluate(
                                  model = model,
                                  training_set = training_set,
                                  validation_set = validation_set,
                                  n_forecasts = n_forecasts,
                                  series_list = True,
                                  model_name = name)

    results_list_gscv[f'{name}_results'] = result
    metrics_list_gscv[f'{name}_metrics']  = {'WAPE':np.mean(result['wape']) ,'Training Time': result['Training Time'],'Forecast Time': result['Forecast Time'], 'split': n_split}

In [ ]:
# lightgbm just for now
'''
model_lgbm = create_lightgbm_model(lags = [-1,-24,-120,-168], output_chunk_length=2, max_depth=3,
                       n_estimators=50, learning_rate=0.1, verbose=-1)
i = 245
plot_splits_gscv = []
pred_gscv = []
for training_set, validation_set in zip(train_splits_gscv, validation_splits_gscv):
  n_forecasts=len(validation_set[1])

  results = fit_predict_and_evaluate(
                        model=model_lgbm,
                        training_set=training_set,
                        validation_set=validation_set,
                        n_forecasts=n_forecasts,
                        series_list=True,
                        model_name='LightGBM')

  pred_gscv.append(results['forecast'])
  splits = [training_set[i], validation_set[i],results['forecast'][i]]
  plot_splits_gscv.append(splits)
  '''
  print(' ')

In [ ]:
#@title Purged Group Cross Validation

pgscv = PurgedGroupTimeSeriesSplit(n_splits=n_splits,group_gap=1,max_train_group_size=25,max_test_group_size=4)
day_groups = df['TimeSeries_id'].unique()
train_split = []
validation_split = []
for i in day_groups:
  group_data = df[df['TimeSeries_id'] == i].reset_index(drop=True)

  X_group = group_data
  sub_groups = group_data['Group']

  train_i = []
  val_i = []

  for train_idx, test_idx in pgscv.split(X_group, groups=sub_groups):
    train, val = ts_hourly_scaled[i][train_idx[0]:train_idx[-1]], ts_hourly_scaled[i][test_idx[0]:test_idx[-1]]

    train_i.append(train)
    val_i.append(val)
  train_split.append(train_i)
  validation_split.append(val_i)


# Splitting the train and test data splits into lists, one for each split
train_splits_pgscv = [[x for x in group] for group in zip(*train_split)]
validation_splits_pgscv =  [[x for x in group] for group in zip(*validation_split)]

results_list_pgscv = {}
metrics_list_pgscv = {}

for name, model in models.items():

  for training_set, validation_set, n_split in zip(train_splits_pgscv, validation_splits_pgscv, range(n_splits)):
    n_forecasts = len(validation_set[1])
    print('\n',name)
    result = fit_predict_and_evaluate(
                                  model = model,
                                  training_set = training_set,
                                  validation_set = validation_set,
                                  n_forecasts = n_forecasts,
                                  series_list = True,
                                  model_name = name)

    results_list_pgscv[f'{name}_results'] = result
    metrics_list_pgscv[f'{name}_metrics']  = {'WAPE':np.mean(result['wape']) ,'Training Time': result['Training Time'],'Forecast Time': result['Forecast Time'], 'split': n_split}

In [ ]:
'''
model_lgbm = create_lightgbm_model(lags = [-1,-24,-120,-168], output_chunk_length=2, max_depth=3,
                       n_estimators=50, learning_rate=0.1, verbose=-1)
i = 245
plot_splits_pgscv = []
pred_pgscv = []
for training_set, validation_set in zip(train_splits_pgscv, validation_splits_pgscv):
  n_forecasts=len(validation_set[1])+24

  results = fit_predict_and_evaluate(
                        model=model_lgbm,
                        training_set=training_set,
                        validation_set=validation_set,
                        n_forecasts=n_forecasts,
                        series_list=True,
                        model_name='LightGBM')

  pred_pgscv.append(results['forecast'])
  splits = [training_set[i], validation_set[i],results['forecast'][i]]
  plot_splits_pgscv.append(splits)
'''
print(' ')